In [15]:
import os
import glob
from PIL import Image
from __future__ import division

In [2]:
# Directories
frames_dir = '/Users/zal/CMU/Spring2017/16824/HW/HW1/Videos/Office_Gines/frames/'
processed_frames_dir = '/Users/zal/CMU/Spring2017/16824/HW/HW1/Videos/Office_Gines/processed_frames/'
padded_proc_frames_dir = '/Users/zal/CMU/Spring2017/16824/HW/HW1/Videos/Office_Gines/padded_processed_frames/'
cropped_frames_dir = '/Users/zal/CMU/Spring2017/16824/HW/HW1/Videos/Office_Gines/cropped_frames/'
resized_frames_dir = os.path.join(processed_frames_dir, 'resized')
comparison_frames_dir = '/Users/zal/CMU/Spring2017/16824/HW/HW1/Videos/Office_Gines/comparison_frames/'

# Constants
img_ext = '.png'
out_dim = 512
padding_size = 200
black_rgb = (0,0,0)

In [6]:
def get_img_list(imgs_dir, img_ext):
    if img_ext[0] != '.':
        img_ext = '.' + img_ext
    img_regex = os.path.join(imgs_dir, ("*"+img_ext))
    img_list = []
    for img in glob.glob(img_regex):
        img_list.append(img)
    return img_list

In [4]:
def resize_dir_imgs(imgs_dir, size):
    '''@size: should be a tuple of (width, height)'''
    pass

In [5]:
def crop_center(img, new_width, new_height):
    width, height = img.size   # Get dimensions
    
    left = (width - new_width)/2
    top = (height - new_height)/2
    right = (width + new_width)/2
    bottom = (height + new_height)/2
    
    res = img.crop((left, top, right, bottom))
    
    return res

Test what is the size of the images

In [19]:
orig_frames_list = get_img_list(frames_dir, img_ext)
print len(orig_frames_list)
frame_1 = Image.open(orig_frames_list[0])
orig_height, orig_width = frame_1.height, frame_1.width
print orig_height, orig_width

460
1920 1080
512 1920
910


In [4]:
frames_regex = os.path.join(frames_dir, ('*'+img_ext))
for img_path in glob.glob(frames_regex):
    img_dir, img_filename = os.path.split(img_path)
    img = Image.open(img_path)
    min_dim = min(img.height, img.width)
    cropped_img = crop_center(img, min_dim, min_dim)
    cropped_img = cropped_img.resize((out_dim, out_dim), Image.BILINEAR)
    cropped_img_p
    ath = os.path.join(cropped_frames_dir, img_filename)
    
    cropped_img.save(cropped_img_path)

In [8]:
proc_frames_regex = os.path.join(processed_frames_dir, ('*.png'))
for img_path in glob.glob(proc_frames_regex):
    img_dir, img_filename = os.path.split(img_path)
    img = Image.open(img_path)
    resized_img = img.resize((out_dim, out_dim), Image.BILINEAR)
    resized_img_path = os.path.join(resized_frames_dir, img_filename)
    resized_img.save(resized_img_path)

In [23]:
def concatenate_images(im1, im2):
    images = map(Image.open, [im1, im2])
    widths, heights = zip(*(i.size for i in images))
    
    total_width = sum(widths)
    max_height = max(heights)
    
    new_im = Image.new('RGB', (total_width, max_height))
    x_offset = 0
    for im in images:
        new_im.paste(im, (x_offset, 0))
        x_offset += im.size[0] # width
    
    return new_im

In [27]:
def set_background(img, bg_size, bg_color):
    '''
    bg_size: tuple with width and height
    bg_color: tuple with (R,G,B) values    
    '''
    img_w, img_h = img.size
    background = Image.new('RGB', bg_size, bg_color)
    bg_w, bg_h = bg_size
    offset = (int((bg_w - img_w)/2), int((bg_h-img_h)/2))
    background.paste(img, offset)
    return background

- Video Dimensions: 720 x 1280
- Concat frames dimension: 640 x 1280
- Video frame aperture: 412 x 412

In [21]:
for img_path in glob.glob(os.path.join(processed_frames_dir, '*.png')):
    # Load the image
    img = Image.open(img_path)
    # upsample the image
    proc_frame_ratio = out_dim/orig_width
    proc_frame_width = out_dim
    proc_frame_heigth = int(proc_frame_ratio * orig_height)
    resized_proc_frame = img.resize((proc_frame_width,proc_frame_heigth), Image.BILINEAR)
    # crop to be a squared image of out_dim x out_dim
    cropped_proc_frame = crop_center(resized_proc_frame, out_dim, out_dim)
    # save file
    _, img_name = os.path.split(img_path)
    padded_path = os.path.join(padded_proc_frames_dir, img_name)
    cropped_proc_frame.save(padded_path)

In [28]:
# Build tuples of frames
cropped_frames = []
for img in glob.glob(os.path.join(cropped_frames_dir, "*.png")):
    cropped_frames.append(img)
processed_frames = []
for img in glob.glob(os.path.join(padded_proc_frames_dir, "*.png")):
    processed_frames.append(img)

video_size = (1280, 720)
video_fit_size = (640, 1280)
for pair in zip(cropped_frames, processed_frames):
    # Concatenate the frames
    concat_img = concatenate_images(pair[0], pair[1])
    
    # Resize into 720p
    video_fit_h, video_fit_w = video_fit_size
    video_fit_img = concat_img.resize((video_fit_w, video_fit_h), Image.BILINEAR)

    # Add black background
    video_frame = set_background(video_fit_img, video_size, black_rgb)
    
    # Store the concatenated frames
    _, frame_name = os.path.split(pair[0])
    frame_path = os.path.join(comparison_frames_dir, frame_name)
    video_frame.save(frame_path)

Convert frames to video:

```
ffmpeg -start_number 0 -i "/Users/zal/CMU/Spring2017/16824/HW/HW1/Videos/Office_Gines/comparison_frames/%09d.png" -c:v libx264 -vf "fps=30,format=yuv420p" comparison.mp4
```
